# Extra Credit

For upto an additional 15 extra credit points on the midterm exam, you can write another function that applies additional criteria on top of what the midterm already did.  To earn the addition points, you need write a function called `summarize_results(patients)` to take the output of your  `process_people()` function to apply a new business rule and summarize the results.

* Your function should take the dictionary created by `process_people()` as it's one input parameter: `patients`.
* If the PRIEST probably is less than 10%, change the `hospital` and `address` to both have a value of `'Home'` indicating patients with a lower probability of an adverse outcome can just go home rather than be admitted to a hospital.
* Summarize those results and count how many patients are now being sent to each location.  Your result should have the following format:
```
{
    "Home": 123,
    "Hospital A": 10,
    "Hospital B": 20,
    "Hospital C": 30
}
```

As always, your function needs good docstrings; but does not require any doctests in this case.

In [5]:
def priest(sex, age, rspRt, oxSt, hrtRt, sybp, tmp, alrtns, inspOyg, sts):
    """
    (String, integer, integer, integer, integer, integer, float, String, String, String) -> float)

    Priest function is calculated using risk percentage as a predictor of
    undesirable effects in adults who have presented to ED with presumed COVID-19.

    Priest function expects sex, alrtns, inspOxy and sts in String and remaining all are in float.

    >>> priest("male",65,10,0.96,135,91,38.1,"ALERT","AIR","unrestricted normal activity")
    0.29

    >>> priest("Female",18,10,0.96,90,230,38.1,"ALERT","AIR","limited self-care")
    0.22

    >>> priest("Female",40,8,0.96,38,115,36.3,"ALERT","AIR","limited strenuous activity, can do light activity")
    0.18

    """

    sex = sex.lower()
    alrtns = alrtns.lower()
    inspOyg = inspOyg.lower()
    sts = sts.lower()
    count = 0

    if sex=="male":
        count += 1

    if age >= 50 and age<=65:
        count = count + 2
    elif age >= 66 and age<=80:
        count = count + 3
    elif age > 80:
        count = count + 4

    if rspRt > 24:
        count = count +  3
    elif rspRt >= 21 and rspRt <= 24:
        count = count +  2
    elif rspRt >= 9 and rspRt <= 11:
        count = count +  1
    elif rspRt < 9:
        count = count +  3

    if oxSt < 0.92:
        count = count +  3
    elif oxSt >= 0.94 and oxSt <= 0.95:
        count = count +  1
    elif oxSt >= 0.92 and oxSt <= 0.93:
        count = count +  2

    if hrtRt > 130:
        count = count +  3
    elif hrtRt >= 111 and hrtRt <= 130:
        count = count +  2
    elif hrtRt >= 91 and hrtRt <= 110:
        count = count +  1
    elif hrtRt < 41:
        count = count +  3
    elif hrtRt >= 41 and hrtRt <= 50:
        count = count +  1

    if sybp > 219:
        count = count +  3
    elif sybp >= 101 and sybp <= 110:
        count = count +  1
    elif sybp >= 91 and sybp <= 100:
        count = count +  2
    elif sybp < 91:
        count = count +  3

    if tmp > 39.0:
        count = count +  2
    elif tmp >= 38.1 and tmp <= 39.0:
        count = count +  1
    if tmp < 35.1:
        count = count +  3
    elif tmp >= 35.1 and tmp <= 36.0:
        count = count +  1

    if alrtns == "confused or not alert":
        count = count +  3

    if inspOyg == "supplemental oxygen":
        count = count +  2

    if sts == "limited self-care":
        count = count +  3
    elif sts == "limited activity, can self-care":
        count = count +  2
    if sts == "limited strenuous activity, can do light activity":
        count = count +  1
    elif sts == "bed/chair bound, no self-care":
        count = count +  4

    riskPtg = [0.01, 0.01, 0.02, 0.02, 0.03, 0.09, 0.15, 0.18, 0.22, 0.26, 0.29, 0.34, 0.38, 0.46, 0.47, 0.49, 0.55, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,0.99]
    count = min(count, len(riskPtg) - 1)
    return riskPtg[count]
import json
import requests
import csv


def find_hospital(age, sex, riskPerc):
    """
    (integer, String, float) -> String

    Find_hospital function Use the age, sex and risk percentage for finding a hospital. It
    expects age, risk percentage in float and sex in String and produces hospital name in String.

     find_hospital function find hospital on API by passing age, sex and rper as query parameters to the url
     using requests module


    >>> find_hospital(66, "female", 0.18)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(18, "Male", 0.26)
    'Southwest Hospital and Medical Center'

    >>> find_hospital(40, "FEMALE", 0.09)
    'Select Specialty Hospital - Northeast Atlanta'

    """
    sex = sex.lower()
    apiurl = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"
    apidata = requests.get(apiurl, params={"age": age, "sex": sex, "risk_pct": riskPerc})
    return (apidata.json())['hospital']

print(find_hospital(18, "Male", 0.26))
def get_address(hospital):

    """
    (String) -> String

    get_address function Find a hospital's address on the basis of its hospital name from this api data

    get_address function determines the hospital's name as String and returns address in String


    >>> get_address("Wesley Woods Geriatric Hospital")
    '1821 CLIFTON ROAD, NE'

    >>> get_address("Emory Dunwoody Medical Center")
    '4500 NORTH SHALLOWFORD ROAD'

    >>> get_address("Select Specialty Hospital - Northeast Atlanta")
    '1821 CLIFTON ROAD NE'

    """

    apiurl = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"
    apidata = requests.get(apiurl)
    hospital=hospital.upper()
    if hospital in (apidata.json()):
        return (apidata.json())[hospital]["ADDRESS"]
    else:
        return " "

import json
import csv
import requests

def process_people(apiurl):
    """
    (String) -> Dictionary

    process_people function is determining the risk percentage, hospital to be visited and address From the group of patients.

    Process_people function accepts floc as a string and returns dictionary in json format. And also It gathers data in the
    csv file from the API and computes risk percentage, hospital to visit and address of the hospital and append
    each patient with information including all factors and risk percentage,
    The hospital's name and address in json format for the dictionary.

    """

    apidata = requests.get(apiurl)
    content = apidata.text.split("\n")
    fd = csv.reader(content, delimiter='|')
    next(fd)
    emptydict = {}
    for i in fd:
        if not any(field != '' for field in i):
            break
        patient, sex, age, rspRt, oxSt, hrtRt, sybp, tmp, alrtns, inspOyg, sts = i
        riskperc = priest(sex, float(age), float(rspRt), float(oxSt), float(hrtRt), float(sybp), float(tmp), alrtns, inspOyg, sts)
        hospital = find_hospital(int(age), sex, riskperc)
        addressHosp = get_address(hospital)
        data = [sex, float(age), float(rspRt), float(oxSt), float(hrtRt), float(sybp), float(tmp), alrtns, inspOyg, sts, riskperc, hospital, addressHosp]
        emptydict[patient] = data

    return emptydict
"""

 It's comparing the actual API data to processed data.
 From the above methods, you will see whether either of them is equal or not.

"""

apiurl = "https://hds5210-data.s3.amazonaws.com/people_results.json"
processedData = process_people("https://hds5210-data.s3.amazonaws.com/people.psv")
apidata = requests.get(apiurl)
APIdata = apidata.json()
X = json.loads(json.dumps(APIdata))
Y= json.loads(json.dumps(processedData))
if  X == Y:
    print("Actual API Data is same as Processed Data")
else:
    print("They are not Equal")



Southwest Hospital and Medical Center
Actual API Data is same as Processed Data


In [6]:
def summarize_results(patients):
    """
    (Dictionary) -> Dictionary

    Description:
    This function is used to mark a patient as home when the risk probability for that patient is less than 10%.

    parameters:
    (patients)

    This function takes a single parameter as input (process_people output)
    and outputs a dictionary-type result that contains a count of all hospitals.

    Returns:
    A dictionary with a distinct hospital count is returned.

    """

    for i, j in patients.items():
        if j[10]<0.1:
            patients[i][11]="Home"
            patients[i][12]="Home"

    temp = set()

    hspl_lst = {}

    key_iterator = iter(patients.keys())
    key = next(key_iterator, None)

    while key is not None:
      value = patients[key][11]
      temp.add(value)
      key = next(key_iterator, None)


    my_list = list(temp)

    patient = 0
    while patient < len(my_list):
        for key, value in patients.items():
            if my_list[patient] == value[11]:
                hspl_lst[my_list[patient]] =  0
        patient = patient+1


    for val in temp:
        for key, value in patients.items():
            if val == value[11]:
                hspl_lst[val] =  hspl_lst[val] + 1

    return hspl_lst

summarize_results(process_people("https://hds5210-data.s3.amazonaws.com/people.psv"))

{'Emory Dunwoody Medical Center': 5,
 'Southwest Hospital and Medical Center': 16,
 'Childrens Healthcare of Atlanta at Scottish Rite': 1,
 'Home': 24,
 'Hughes Spalding Childrens Hospital': 1,
 'Select Specialty Hospital - Northeast Atlanta': 32,
 'Wesley Woods Geriatric Hospital': 20}